In [77]:
from gensim.models.word2vec import KeyedVectors

w2v = KeyedVectors.load_word2vec_format(r'E:/迅雷下载/GoogleNews-vectors-negative300.bin', binary=True)

In [78]:
w2v['hello'].shape

(300,)

In [72]:
import numpy as np
import pandas as pd

data = pd.read_csv("./DATA/training.csv", encoding='ISO8859-2')


In [75]:
def remove_strip(str):
    return str.replace("\r", "").replace("\n", "").strip()

def get_stop_words(f, encoding='utf-8'):
    stop_words = []
    with open(f, "r", encoding=encoding) as f_stopwords:
        for line in f_stopwords:
            line = remove_strip(line)
            stop_words.append(line.lower())
    stop_words = set(stop_words)
    print("total counts: ", len(stop_words))
    return stop_words

stop_words = get_stop_words("stopwords.txt")
len(stop_words)

total counts:  233


233

In [79]:
def buildWordVector(text):
    words = [w for w in text.replace(",", "").replace(".", "").replace("!", "").split() if w.lower() not in stop_words]
    count = len(words)
    vec = np.zeros(300).reshape((1, 300))
    for word in text:
        try:
            vec += w2v[word].reshape((1, 300))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [85]:
data['comment_text'] = data['comment_text'].apply(lambda doc: buildWordVector(doc))

In [86]:
data_word2vec = pd.DataFrame(data['comment_text'].tolist())
data_word2vec


ValueError: Must pass 2-d input. shape=(127657, 1, 300)

In [ ]:
def read_corpus(fname, tokens_only=False, encoding='utf-8'):
    import gensim
    import smart_open
    with smart_open.open(fname, encoding=encoding) as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])